# 사용함수 정의

In [ ]:
from sqlalchemy import create_engine
from datetime import datetime
from sklearn.model_selection import KFold, cross_val_score
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
import os
import warnings
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import boto3
from plugins import connector

warnings.filterwarnings('ignore')

    
def extract_data(cur, sql):
    cur.execute(sql)
    columns = [desc[0] for desc in cur.description]
    results = cur.fetchall()
    print(f'{len(results)}행 추출 완료')
    df = pd.DataFrame(results)
    df.columns = columns
    print('데이터프레임 생성 완료')
    return df

# REDSHIFT 데이터 추출
데이터 웨어하우스에 적재된 population 데이터와 weather 데이터를 join하여 데이터를 추출하였습니다.

In [ ]:
conn, cur = connector.redshift_connector()

# sql 수정 후에 사용해도 좋음
# 내가 생각하기에 쓸모 없다고 생각하는 데이터는 뺐음
sql = '''
    SELECT
        P.place_id,
        P.area_congest_id,
        P.area_ppltn_min,
        P.area_ppltn_max,
        W.temp,
        W.sensible_temp,
        W.humidity,
        W.wind_dirct,
        W.wind_spd,
        W.precipitation,
        W.uv_index_lvl,
        W.pm25,
        W.pm10,
        W.air_idx_mvl,
        W.created_date,
        DATE_PART('year', W.created_date) AS year,
        DATE_PART('month', W.created_date) AS month,
        DATE_PART('day', W.created_date) AS day,
        DATE_PART('hour', W.created_date) AS hour,
        DATE_PART('minute', W.created_date) AS minute,
        EXTRACT(DOW FROM W.created_date) AS dow
    FROM
        "raw"."population" AS P
    JOIN
        "raw"."weather" AS W ON P.place_id = W.place_id AND P.created_date = W.created_date;
    '''
    
df = extract_data(cur, sql)
df.info()

## 예측에 사용할 수 없는 변수 삭제
- 데이터가 한정되어 있어, year, month는 일년동안 데이터를 쌓지 않는 이상 제대로 된 변수로 사용될 수 없을 것 같아 drop 하였습니다.
- area_ppltn_min, area_ppltn_max, sensible_temp, uv_index_lvl, pm25, pm10, air_idx_mvl 컬럼은 예측 시점의 데이터를 가져오는 것이 불가능하기에 제외하였습니다.
- wind_dirct의 경우 타겟변수와 유의미한 상관관계도 발견되지 않아, 제외하였습니다.

In [ ]:
cannotuse_columns = ['year', 'month', 'area_ppltn_min', 'area_ppltn_max', 'sensible_temp', 'wind_dirct', 'uv_index_lvl', 'pm25', 'pm10', 'air_idx_mvl']
df = df.drop(columns = cannotuse_columns)

## 푸리에특징을 통한 시간연속성 표현(HOUR)
- 간단한 푸리에 변환을 활용하여, hour의 시간연속성을 데이터에 표현하였습니다.

In [ ]:
# 시간 관련 정보 추출
hour = df['hour'].values

# 시간 정보를 하나의 특징으로 합치기
time_feature = hour / 24

# 주기성을 나타내는 푸리에 특징 계산
time_rad = 2 * np.pi * time_feature

fourier_features = np.column_stack([
    np.cos(time_rad), np.sin(time_rad)
])

# 생성된 특징을 데이터프레임에 추가
df['fourier_cos_time'] = fourier_features[:, 0]
df['fourier_sin_time'] = fourier_features[:, 1]

# 기존 시간 관련 컬럼 제거
df = df.drop(columns=['day','hour','minute'])
df


# 휴일컬럼 추가
공휴일인 경우 1의 값을 가집니다. 아닌 경우는 모두 0입니다.

In [ ]:
# 휴일 관련 컬럼을 정의합니다.
holiday_list = joblib.load('C:/last_project/ML/data/holiday.pkl')

# created_date 컬럼을 datetime 형식으로 변환
df['created_date'] = pd.to_datetime(df['created_date'])

# holiday인 값을 디폴트 값 0으로 설정
df['holiday'] = 0

# created_date가 holiday_list에 포함된 날짜인 경우, holiday 컬럼을 1로 변경
df.loc[df['created_date'].dt.strftime('%Y-%m-%d').isin(holiday_list), 'holiday'] = 1

df = df.drop(columns = 'created_date')
df

# 원핫인코딩
범주형 변수인 place_id와 dow(요일)을 원핫인코딩 처리하였습니다. 그 외에는 전부 연속형 변수입니다.(타겟변수인 area_congest_id는 제외)

In [ ]:
onehot_features = ['place_id', 'dow']

# one-hot encoding 수행
df = pd.get_dummies(df, columns=onehot_features, dummy_na=False)
df

# 하이퍼파라미터 튜닝
- 타겟변수의 불균형 문제를 해결하기 위해 SMOTE 기법과, 클래스별 가중치를 비교해본 결과, SMOTE 기법은 과적합 문제로 인해서 새로운 데이터를 잘 예측하지 못하는 특성이 있었습니다.
- 시계열 데이터의 특성보단, hour 피처만 푸리에 변환을 하였고, 나머지 시계열 데이터의 특성은 없앴으며, 클래스의 가중치를 주었습니다.
- 그리드 서치를 활용하여 검증하였습니다.

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE

def compute_class_weights(y):
    class_counts = y.value_counts()
    total_samples = len(y)
    class_weights = {}
    for class_label, count in class_counts.items():
        weight = total_samples / (len(class_counts) * count)
        class_weights[class_label] = weight
    return class_weights

# 특성과 타겟 데이터 분할
X = df.drop(columns='area_congest_id')
y = df['area_congest_id']

# 클래스 가중치 계산
class_weights = compute_class_weights(y)

# 모델 초기화
model = CatBoostClassifier(random_state=42, class_weights=class_weights)

# 하이퍼파라미터 그리드 준비
param_grid = {
    'learning_rate': [0.01, 0.1, 0.5],
    'depth': [3, 5, 7],
    'iterations': [300, 500, 1000],
    'l2_leaf_reg': [1, 3, 5],
    'bagging_temperature': [0.5, 1, 1.5],
    'random_strength': [0.5, 1, 2]
}

# 교차 검증과 하이퍼파라미터 튜닝 수행
kf = KFold(n_splits=3, shuffle=True, random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=kf)
grid_search.fit(X, y)

# 최적의 모델과 하이퍼파라미터 출력
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print("Best Model:", best_model)
print("Best Parameters:", best_params)

# 교차 검증 결과 출력
accuracy_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

# 각 폴드의 정확도 점수 출력
for fold, accuracy in enumerate(accuracy_scores):
    print(f"Fold {fold+1} Accuracy: {accuracy}")

In [ ]:
# from sklearn.metrics import precision_score, recall_score, f1_score

# # 훈련 데이터에 대한 성능 평가
# accuracy_train = accuracy_score(y_resampled, y_pred_train)
# precision_train = precision_score(y_resampled, y_pred_train, average='weighted')  # 다중 클래스 평가를 위해 average='weighted' 사용
# recall_train = recall_score(y_resampled, y_pred_train, average='weighted')  # 다중 클래스 평가를 위해 average='weighted' 사용
# f1_train = f1_score(y_resampled, y_pred_train, average='weighted')  # 다중 클래스 평가를 위해 average='weighted' 사용

# # 테스트 데이터에 대한 성능 평가
# y_pred_test = best_model.predict(X_test)
# accuracy_test = accuracy_score(y_test, y_pred_test)
# precision_test = precision_score(y_test, y_pred_test, average='weighted')  # 다중 클래스 평가를 위해 average='weighted' 사용
# recall_test = recall_score(y_test, y_pred_test, average='weighted')  # 다중 클래스 평가를 위해 average='weighted' 사용
# f1_test = f1_score(y_test, y_pred_test, average='weighted')  # 다중 클래스 평가를 위해 average='weighted' 사용

# # 출력
# print("Best Model Evaluation:")
# print(f"Train Accuracy: {accuracy_train}")
# print(f"Train Precision: {precision_train}")
# print(f"Train Recall: {recall_train}")
# print(f"Train F1 Score: {f1_train}")
# print(f"Test Accuracy: {accuracy_test}")
# print(f"Test Precision: {precision_test}")
# print(f"Test Recall: {recall_test}")
# print(f"Test F1 Score: {f1_test}")

# # 학습 곡선 그리기
# from sklearn.model_selection import learning_curve
# import matplotlib.pyplot as plt

# train_sizes, train_scores, test_scores = learning_curve(best_model, X_resampled, y_resampled, cv=kf, scoring='accuracy', train_sizes=np.linspace(0.1, 1.0, 10))
# train_mean = np.mean(train_scores, axis=1)
# train_std = np.std(train_scores, axis=1)
# test_mean = np.mean(test_scores, axis=1)
# test_std = np.std(test_scores, axis=1)

# plt.figure(figsize=(10, 6))
# plt.plot(train_sizes, train_mean, label='Train Accuracy')
# plt.plot(train_sizes, test_mean, label='Validation Accuracy')
# plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
# plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1)
# plt.xlabel('Training Set Size')
# plt.ylabel('Accuracy')
# plt.title('Learning Curve')
# plt.legend(loc='best')
# plt.show()

In [ ]:
import joblib
from datetime import datetime


# Best Model: <catboost.core.CatBoostClassifier object at 0x0000025F30E7C250>
# Best Parameters: {'depth': 7, 'iterations': 1000, 'learning_rate': 0.5}

now = datetime.now().strftime('%Y%m%d')
# 최적의 모델 저장
joblib.dump(best_model, f'best_model_{now}.pkl')